<a href="https://colab.research.google.com/github/linhvien/Stock-analysis/blob/main/Vietnamese_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

import re
import json
import csv
from io import StringIO
from datetime import datetime, timedelta
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Overview

In [115]:
start_date = '2019-01-01'
end_date = str(datetime.now().strftime('%Y-%m-%d'))
stock = 'BID'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

#Financial data
Loading quarterly financial data of Vietnamese stocks from api vndirect.

In [116]:
class Financialdata:
  url_BS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=1,89,101,411&fromDate={}&toDate={}'
  url_IS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=2,90,102,412&fromDate={}&toDate={}'
  url_CF = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=3,91,103,413&fromDate={}&toDate={}'
  url_ratio = 'https://finfo-api.vndirect.com.vn/v4/ratios?q=code:{}~itemCode:53030,52005,51050,53021,52001,52002,54018,712010,712020,712030,712040~reportDate:{}'
  
  def __init__(self,stock,start_date,end_date,*arg, **karg):
    self.stock = stock
    self.start_date = start_date
    self.end_date = end_date
    self.session = requests.Session()

# Get balance sheet
  def get_BS(self):
    try:
      page = self.session.get(self.url_BS.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

# get income statement
  def get_IS(self):
    try:
      page = self.session.get(self.url_IS.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

# get cashflow statement
  def get_CF(self):
    try:
      page = self.session.get(self.url_CF.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

In [118]:
BS_data = Financialdata(stock,start_date,end_date).get_BS()
BS_data.to_excel(r'BS_{}.xlsx'.format(stock),index=True)
BS_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
"Tiền mặt, vàng bạc, đá quí",11073415000000,10770158000000,12294194000000,10401494000000,11822072000000,9097015000000,14116670000000,12638511000000,10274109000000,9808400000000
Tiền gửi tại NHNN,37588465000000,91672598000000,49434766000000,40335703000000,28062207000000,26782623000000,135255429000000,42191695000000,15351805000000,32963975000000
"Tiền, vàng gửi tại các TCTD khác và cho vay các TCTD khác",145957757000000,85087146000000,85348812000000,116003134000000,107702884000000,121290315000000,54290120000000,130025743000000,141847419000000,112271066000000
Tiền gửi tại các TCTD khác,125444438000000,74269353000000,62192181000000,96756092000000,83243633000000,91778750000000,43718543000000,107992152000000,116909976000000,86902217000000
Cho vay các TCTD khác,20647195000000,10957492000000,23295457000000,19391253000000,24604400000000,29658771000000,10717769000000,22143226000000,25048728000000,25480379000000


In [119]:
IS_data = Financialdata(stock,start_date,end_date).get_IS()
IS_data.to_excel(r'IS_{}.xlsx'.format(stock),index=True)
IS_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
Chi phí lãi và các khoản chi phí tương tự,13241606000000,13228171000000,15394359000000,15590975000000,16518543000000,17386826000000,17057425000000,16766310000000,15998300000000,14947382000000
Thu nhập lãi thuần,12929461000000,10830069000000,10565402000000,9144289000000,6939043000000,9148651000000,9596941000000,8751685000000,9101188000000,8545052000000
Thu nhập từ hoạt động dịch vụ,2702988000000,2217213000000,2576222000000,2160529000000,2055788000000,1822199000000,2362350000000,1943319000000,1958394000000,1627188000000
Chi phí hoạt động dịch vụ,961339000000,782881000000,983690000000,792731000000,842724000000,736287000000,1097291000000,892993000000,866452000000,750722000000
Thu nhập thuần từ hoạt động dịch vụ,1741649000000,1434332000000,1592532000000,1367798000000,1213064000000,1085912000000,1265059000000,1050326000000,1091942000000,876466000000


In [ ]:
CF_data = Financialdata(stock,start_date,end_date).get_CF()
CF_data.to_excel(r'CF_{}.xlsx'.format(stock),index=True)
CF_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
Chi phí khấu hao tài sản cố định,547756789868,499689193833,545413737797,551231227788,556761193198,555460462969,502928809106,494141498125,478654208826,472348259477
Phân bổ lợi thế thương mại,61385028806,61385028806,349960385329,123278040202,67545449401,67364700041,16987979595,16987979595,16987979595,16987979595
"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",-3508982145,3789250096,43717904388,-2000594719,6782636573,1003914771,1003332731,-748428436,2076193560,5001735507
"Lãi, lỗ chênh lệch tỷ giá hối đoái chưa thực hiện",-3585620065,-1281139009,17776990028,-14810127185,-70182197587,67852556191,7871927410,-678177168,468890998,-1204432181
Lãi/(lỗ) từ thanh lý tài sản cố định,16223729056,5187526637,14082316639,9964216884,10375293005,8717274881,6348242528,13281469680,10580680034,13753677997


#Full stock and industry

In [ ]:
# Get industry list from vietstock
r = requests.get('https://finance.vietstock.vn/chi-so-nganh.htm',headers=headers)
soup = bs(r.content)
def Getsectorname():
  sectors = soup.select('a[href^="/nganh"]')
  all_sector = []
  for i in sectors:
    sector_name = i.get('title')
    if sector_name not in all_sector:
      all_sector.append(sector_name)
  return all_sector

In [ ]:
Getsectorname()

['Bán buôn',
 'Bảo hiểm',
 'Bất động sản',
 'Chứng khoán',
 'Công nghệ và thông tin',
 'Bán lẻ',
 'Chăm sóc sức khỏe',
 'Khai khoáng',
 'Ngân hàng',
 'Nông - Lâm - Ngư',
 'SX Thiết bị, máy móc',
 'SX Hàng gia dụng',
 'Sản phẩm cao su',
 'SX Nhựa - Hóa chất',
 'Thực phẩm - Đồ uống',
 'Chế biến Thủy sản',
 'Vật liệu xây dựng',
 'Tiện ích',
 'Vận tải - kho bãi',
 'Xây dựng',
 'Dịch vụ lưu trú, ăn uống, giải trí',
 'SX Phụ trợ',
 'Thiết bị điện',
 'Dịch vụ tư vấn, hỗ trợ',
 'Tài chính khác']

In [ ]:
def GetURL():
    sectors = soup.select('a[href^="/nganh"]')
    all_sector_URL = []
    for sector in sectors:
        sector_ID = sector.get('href')
        sector_URL = 'https://finance.vietstock.vn' + sector_ID
        if sector_URL not in all_sector_URL:
            all_sector_URL.append(sector_URL)
    return all_sector_URL

In [ ]:
url = GetURL()
url

['https://finance.vietstock.vn/nganh/1-ban-buon.htm',
 'https://finance.vietstock.vn/nganh/2-bao-hiem.htm',
 'https://finance.vietstock.vn/nganh/3-bat-dong-san.htm',
 'https://finance.vietstock.vn/nganh/5-chung-khoan.htm',
 'https://finance.vietstock.vn/nganh/6-cong-nghe-va-thong-tin.htm',
 'https://finance.vietstock.vn/nganh/7-ban-le.htm',
 'https://finance.vietstock.vn/nganh/8-cham-soc-suc-khoe.htm',
 'https://finance.vietstock.vn/nganh/10-khai-khoang.htm',
 'https://finance.vietstock.vn/nganh/11-ngan-hang.htm',
 'https://finance.vietstock.vn/nganh/12-nong-lam-ngu.htm',
 'https://finance.vietstock.vn/nganh/15-sx-thiet-bi-may-moc.htm',
 'https://finance.vietstock.vn/nganh/16-sx-hang-gia-dung.htm',
 'https://finance.vietstock.vn/nganh/17-san-pham-cao-su.htm',
 'https://finance.vietstock.vn/nganh/18-sx-nhua-hoa-chat.htm',
 'https://finance.vietstock.vn/nganh/19-thuc-pham-do-uong.htm',
 'https://finance.vietstock.vn/nganh/20-che-bien-thuy-san.htm',
 'https://finance.vietstock.vn/nganh/21

In [ ]:
#Get full stocks along with industry
full_stock =[]
for i in url:
  try:
    industry = pd.read_html(requests.get(i,headers=headers).text)
    industry = pd.DataFrame(industry[1])
    full_stock.append(industry)
  except:
    pass
symbols = pd.concat(full_stock, axis=0, ignore_index=True)
symbols['Giá 1 ngày'] = symbols['Giá 1 ngày'].map(lambda x: x.split(' ')[0].replace(',',''))
symbols['Giá 1 ngày'] = symbols['Giá 1 ngày'].astype(int)
symbols = symbols.drop(columns=['STT','Sàn'])
symbols.rename(columns = {'Mã CK':'Stock', 'Ngành cấp 3':'Sector',
               'KLCPLH':'Share numbers', 'Giá 1 ngày':'Real-time price','Giá 5 ngày': 'Average 5-day price'}, inplace = True)
symbols.tail()

,Stock,Sector,Share numbers,Real-time price,Average 5-day price
735,VCM,Dịch vụ việc làm,3000000,15000,16500.0
736,VLA,Dịch vụ lập trình máy tính,1080000,16200,17000.0
737,VNC,"Các dịch vụ chuyên môn, khoa học và kỹ thuật khác",10499560,33000,32000.0
738,OGC,Các hoạt động đầu tư tài chính khác,299999999,6190,7450.0
739,TVC,Các hoạt động đầu tư tài chính khác,103610670,17000,18500.0


In [ ]:
#Cách khác
#Get real time price for stock
def getrealtimeprice(stock):
  r = requests.get('https://finance.vietstock.vn/{}/thong-ke-giao-dich.htm'.format(stock),headers=headers)
  soup = bs(r.content)
  price = soup.select('h2#stockprice span')[0].text
  price = price.replace(',', '')
  return int(price)

#Financial ratio analysis
Create function to calculate financial ratio for stock list

In [ ]:
stocklist=[] 
def FinRatio(stocklist):
  FinRatio = {}
  for stock in stocklist:
    BS = Financialdata(stock,start_date,end_date).get_BS()
    IS = Financialdata(stock,start_date,end_date).get_IS()
    CF = Financialdata(stock,start_date,end_date).get_CF()
    price = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
    KLCPLH = int(symbols.loc[symbols['Stock'] == stock, 'Share numbers'].to_string(index=False).strip())
    market_cap = KLCPLH * price

    FinRatio[stock] = {}
    FinRatio[stock]['market_cap'] = market_cap
    FinRatio[stock]['Trailing PE'] = market_cap/(IS.loc['Lợi nhuận sau thuế thu nhập doanh nghiệp'][0:4].sum())
    FinRatio[stock]['Price/FCF'] = market_cap/ (CF.loc['Lưu chuyển tiền thuần từ hoạt động kinh doanh'][0]+CF.loc['Mua sắm TSCĐ'][0]+CF.loc['Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác'][0])
    FinRatio[stock]['ROE'] = IS.loc['Doanh thu thuần'].mean()/BS.loc['Vốn chủ sở hữu'].mean()
    FinRatio[stock]['ROA'] = IS.loc['Doanh thu thuần'].mean()/BS.loc['TỔNG CỘNG TÀI SẢN'].mean()
    FinRatio[stock]['EPS 4 latest quart'] = IS.loc['Lãi cơ bản trên cổ phiếu'][0:4].sum()
    FinRatio[stock]['Operating margin'] = IS.loc['Lợi nhuận thuần từ hoạt động kinh doanh'][0:4].mean()/IS.loc['Doanh thu thuần'][0:4].mean()
    FinRatio[stock]['Operating cashflow margin'] = CF.loc['Lưu chuyển tiền thuần từ hoạt động kinh doanh'][0:4].mean()/IS.loc['Doanh thu thuần'][0:4].mean()

  df = pd.DataFrame.from_dict(FinRatio,orient='index')
  return df

In [126]:
#CHú ý: Dành cho các tổ chức tài chính
stocklist=[] 
def FinRatio_finance(stocklist):
  FinRatio = {}
  for stock in stocklist:
    BS = Financialdata(stock,start_date,end_date).get_BS()
    IS = Financialdata(stock,start_date,end_date).get_IS()
    CF = Financialdata(stock,start_date,end_date).get_CF()
    price = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
    KLCPLH = int(symbols.loc[symbols['Stock'] == stock, 'Share numbers'].to_string(index=False).strip())
    market_cap = KLCPLH * price

    FinRatio[stock] = {}
    FinRatio[stock]['market_cap'] = market_cap
    FinRatio[stock]['Trailing PE'] = market_cap/(IS.loc['Lợi nhuận sau thuế thu nhập doanh nghiệp'][0:4].sum())
    FinRatio[stock]['ROE'] = IS.loc['Thu nhập lãi thuần'].mean()/BS.loc['Vốn chủ sở hữu'].mean()
    FinRatio[stock]['ROA'] = IS.loc['Thu nhập lãi thuần'].mean()/BS.loc['TỔNG CỘNG NGUỒN VỐN'].mean()
    FinRatio[stock]['EPS 4 latest quart'] = IS.loc['Lãi cơ bản trên cổ phiếu'][0:4].sum()

  df = pd.DataFrame.from_dict(FinRatio,orient='index')
  return df


##Portfolio analysis

In [ ]:
portfolio = ['VNM','VIC']
FinRatio(portfolio)

,market_cap,Trailing PE,Price/FCF,ROE,ROA,EPS 4 latest quart,Operating margin,Operating cashflow margin
VNM,186633021238500,17.227298,69.980204,0.461767,0.313625,4572,0.221353,0.160257
VIC,329151079421500,73.870652,-178.898999,0.251776,0.080582,476,0.105314,0.090409


##Peers analysis

In [ ]:
# Get stock list in the same sector with selected stock
industry = symbols.loc[symbols['Stock'] == stock, 'Sector'].to_string(index=False).strip()
stocklist = symbols['Stock'].loc[symbols['Sector'] == industry].tolist()
# Compare ratios between stocks in the same sector
df = FinRatio(stocklist)

Determine if any stock is under/over valued with PE

In [109]:
#Add current price into data frame
df_price={}
for stock in stocklist:
  df_price[stock] = {}
  df_price[stock]['Current Price'] = int(symbols.loc[symbols['Stock'] == stock, 'Real-time price'].to_string(index=False).strip())
df_price = pd.DataFrame.from_dict(df_price,orient='index')
a = df.join(df_price, how='outer')
#Calculate the mean PE ratio
PE_mean = a['Trailing PE'].mean()
#Calculate the fair market value
a['Fair Market value']= PE_mean * a['EPS 4 latest quart']
#Calculate over/under valued ratio
a['Over_Under ratio'] = a['Current Price']/a['Fair Market value']
#Show a label of under valued and over valued stocks
a['Value_label'] = np.where(a['Over_Under ratio']<1.0, 'Under Valued', 'Fair or Over Valued')
a.insert(0, 'Value_label',a.pop('Value_label'))
a

,Value_label,market_cap,Trailing PE,Price/FCF,ROE,ROA,EPS 4 latest quart,Operating margin,Operating cashflow margin,Current Price,Fair Market value,Over_Under ratio
AGG,Under Valued,3541724695600,7.897975,5.111849,0.154683,0.038605,5093,0.203328,0.699587,42800,164965.594741,0.259448
API,Under Valued,1596540000000,27.857920,-20.302371,0.256949,0.051637,1618,0.119954,-0.142817,45100,52408.076240,0.860554
BCE,Under Valued,490000000000,9.535207,5.997537,0.318050,0.120149,1390,0.140351,0.453116,14000,45023.007400,0.310952
CDC,Under Valued,327631868400,17.583376,-2.702973,0.378840,0.118297,917,0.051480,-0.238835,14900,29702.228623,0.501646
CEO,Fair or Over Valued,2573399850000,-16.279716,-320.571298,0.187452,0.078589,0,-0.031239,-0.189270,10000,0.000000,inf
CIG,Fair or Over Valued,223933623700,-7.915757,689.515165,0.012321,0.003753,0,-13.935814,-0.197606,7100,0.000000,inf
CKG,Under Valued,1975869754950,14.005270,-9.867257,0.398821,0.063262,2558,0.153651,0.204443,23950,82855.289878,0.289058
D11,Under Valued,176247858500,5.425097,9.089401,0.581124,0.104398,3960,0.214427,-0.522976,26900,128266.985112,0.209719
D2D,Fair or Over Valued,1603766326000,7.080710,-41.564618,0.160793,0.062652,0,0.723423,-0.736308,53000,0.000000,inf
DIG,Fair or Over Valued,13171619822000,19.383432,-70.761789,0.139612,0.063821,0,0.119557,0.688917,31000,0.000000,inf


In [112]:
#Show a list of stocks that are considered under valued
a[a['Value_label'] == 'Under Valued'].index

Index(['AGG', 'API', 'BCE', 'CDC', 'CKG', 'D11', 'DXG', 'FIR', 'HLD', 'HUT',
       'IJC', 'ITC', 'KDH', 'NBB', 'NLG', 'NTL', 'NVL', 'PDR', 'TCH', 'TDC',
       'VHM', 'VPI'],
      dtype='object')

#Historical data

In [ ]:
def convert_date(text, date_type = '%Y-%m-%d'):
  return datetime.strptime(text, date_type)
  
def convert_text_dateformat(text, origin_type = '%Y-%m-%d', new_type = '%Y-%m-%d'):
  return convert_date(text, origin_type).strftime(new_type)

def split_change_col(text):
    return re.sub(r'[\(|\)%]', '', text).strip().split()

class DataLoadProto():
    def __init__(self, symbols, start, end, *arg, **karg):
        self.symbols = symbols
        self.start = convert_text_dateformat(start, new_type = '%d/%m/%Y')
        self.end = convert_text_dateformat(end, new_type = '%d/%m/%Y')

class DataLoaderCAFE(DataLoadProto):
    def __init__(self, symbols, start, end, *arg, **karg):
        self.symbols = symbols
        self.start = start
        self.end = end
        super(DataLoaderCAFE, self).__init__(symbols, start, end)

    def download(self):
        stock_datas = []
        if not isinstance(self.symbols, list):
            symbols = [self.symbols]
        else:
            symbols = self.symbols

        for symbol in symbols:
            stock_datas.append(self.download_one(symbol))

        data = pd.concat(stock_datas, axis=1)
        return data

    def download_one(self, symbol):
        stock_data = pd.DataFrame(columns=['date', 'change_perc1', 'change_perc2',
                                           'open', 'high', 'low', 'close',
                                           'avg', 'volume_match', 'volume_reconcile'])

        for i in range(1000):
            stock_slice_batch = self.download_batch(i + 1, symbol)
            stock_data = pd.concat([stock_data, stock_slice_batch], axis=0)
            try:
                date_end_batch = stock_slice_batch.date.values[-1]
            except:
                # start date is holiday or weekend
                break
            is_touch_end = convert_date(self.start, '%d/%m/%Y') == convert_date(date_end_batch, '%d/%m/%Y')
            # logging.info('batch: {}; start date out range: {}; date_end_batch: {}'.format(i + 1, is_touch_end, date_end_batch))
            if is_touch_end:
                break

        stock_data['change_perc1'], stock_data['change_perc2'] = stock_data['change_perc'].apply(split_change_col).str
        if 'change_perc' in stock_data.columns:
            stock_data.pop('change_perc')
        if 'avg' in stock_data.columns:
            stock_data.pop('avg')
            stock_data = stock_data.set_index('date').apply(pd.to_numeric, errors='coerce')
            stock_data.index = list(map(lambda text: convert_date(text, date_type='%d/%m/%Y'), stock_data.index))
            stock_data.index.name = 'date'
            stock_data = stock_data.sort_index()
            stock_data.fillna(0, inplace=True)
            stock_data['volume'] = stock_data.volume_match + stock_data.volume_reconcile

        # Create multiple columns
        iterables = [stock_data.columns.tolist(), [symbol]]
        mulindex = pd.MultiIndex.from_product(iterables, names=['Attributes', 'Symbols'])
        stock_data.columns = mulindex


        logging.info('data {} from {} to {} have already cloned!' \
                     .format(symbol,
                             convert_text_dateformat(self.start, origin_type = '%d/%m/%Y', new_type = '%Y-%m-%d'),
                             convert_text_dateformat(self.end, origin_type='%d/%m/%Y', new_type='%Y-%m-%d')))

        return stock_data

    def download_batch(self, id_batch, symbol):
        form_data = {'ctl00$ContentPlaceHolder1$scriptmanager':'ctl00$ContentPlaceHolder1$ctl03$panelAjax|ctl00$ContentPlaceHolder1$ctl03$pager2',
                       'ctl00$ContentPlaceHolder1$ctl03$txtKeyword':symbol,
                       'ctl00$ContentPlaceHolder1$ctl03$dpkTradeDate1$txtDatePicker':self.start,
                       'ctl00$ContentPlaceHolder1$ctl03$dpkTradeDate2$txtDatePicker':self.end,
                       '__EVENTTARGET':'ctl00$ContentPlaceHolder1$ctl03$pager2',
                       '__EVENTARGUMENT':id_batch,
                       '__ASYNCPOST':'true'}
        url = URL_CAFE+symbol+"-1.chn"
        r = requests.post(url, data = form_data, headers = headers, verify=False)
        soup = BeautifulSoup(r.content, 'html.parser')
        print(id_batch)
        table = soup.find('table')
        stock_slice_batch = pd.read_html(str(table))[0].iloc[2:, :12]

        stock_slice_batch.columns = ['date', 'adjust', 'close', 'change_perc', 'avg',
                        'volume_match', 'value_match', 'volume_reconcile', 'value_reconcile',
                        'open', 'high', 'low']

        return stock_slice_batch


#Ben Graham Stock Screening
This method of Ben Graham is often used very effectively when the market is in a bear trend or the stock is under accumulation.

Phương pháp này của Ben Graham thường sử dụng rất hiệu quả trong 1 thị trường giá xuống hoặc thị trường tích lũy ở vùng đáy.

<b> Chú ý: Bộ lọc theo phương pháp của Ben Graham không áp dụng cho cổ phiếu thuộc lĩnh vực tài chính như ngân hàng, công ty bảo hiểm hay công ty chứng khoán.



Stock screening according to NCAV:
> NCAV = Current Assets – Total Liabilities

- 1. P/NCAV < 1
- 2. 3M Average Volume > 500 million VND
- 3. Arranging the P/NCAV ratio in ascending order

Stock screening according to NNWC:
> NNWC = Cash & Cash equivalents + Short-term investment + 0.75 x Account Receivables + 0.5 x Inventory – Total Liabilities

- 1. P/NNWC < 1
- 2. 3M Average Volume > 500 million VND
- 3. Arranging the P/NNWC ratio in ascending order

#Warren Buffett Stock Screening
Need data on all stocks in Vietnam market
- 1. Market capitalization > 50 million USD
- 2. Operating income > 0 for 5 consecutive years
- 3. ROE greater than 15% in the last 2 years and accumulated for 12 months
- 4. P/FCF ratio (Price/Free Cash flow) is among the lowest 30% in the whole market
- 5. Operating income margin > Operating income margin Median
- 6. Net income margin > Net income margin Median

#Philip Fisher Stock Screening
Tập trung vào những cổ phiếu tăng trưởng
- 1. Sales 5Y CAGR > Sales 5Y CAGR Median
- 2. Doanh thu tăng trưởng trong 2 năm gần nhất
- 3. Net income margin > Net income margin Median
- 4. Tỷ lệ PEG (5Y Growth) < 0.5
- 5. Sắp xếp tỷ lệ PEG theo thứ tự tăng dần


#CANSLIM Stock Screening
- C: Current Quaterly Earnings Per Share:
  
  + Tăng trưởng EPS quý gần nhất và quý gần liền kề phải đạt tối thiểu 20% – 25% so với quý cùng kỳ

- 